# Scrape scripts to get data from Ourlads and ESPN

## Set up the environments to enable Selenium to run on Databricks.

In [ ]:
dbutils.fs.mkdirs("dbfs:/databricks/scripts/")
dbutils.fs.put("/databricks/scripts/selenium-install.sh","""
#!/bin/bash
%sh
LAST_VERSION="https://www.googleapis.com/download/storage/v1/b/chromium-browser-snapshots/o/Linux_x64%2FLAST_CHANGE?alt=media"
VERSION=$(curl -s -S $LAST_VERSION)
if [ -d $VERSION ] ; then
  echo "version already installed"
  exit
fi
 
rm -rf /tmp/chrome/$VERSION
mkdir -p /tmp/chrome/$VERSION
 
URL="https://www.googleapis.com/download/storage/v1/b/chromium-browser-snapshots/o/Linux_x64%2F$VERSION%2Fchrome-linux.zip?alt=media"
ZIP="${VERSION}-chrome-linux.zip"
 
curl -# $URL > /tmp/chrome/$ZIP
unzip /tmp/chrome/$ZIP -d /tmp/chrome/$VERSION
 
URL="https://www.googleapis.com/download/storage/v1/b/chromium-browser-snapshots/o/Linux_x64%2F$VERSION%2Fchromedriver_linux64.zip?alt=media"
ZIP="${VERSION}-chromedriver_linux64.zip"
 
curl -# $URL > /tmp/chrome/$ZIP
unzip /tmp/chrome/$ZIP -d /tmp/chrome/$VERSION
 
mkdir -p /tmp/chrome/chrome-user-data-dir
 
rm -f /tmp/chrome/latest
ln -s /tmp/chrome/$VERSION /tmp/chrome/latest
 
# to avoid errors about missing libraries
sudo apt-get update
sudo apt-get install -y libgbm-dev
""", True)
display(dbutils.fs.ls("dbfs:/databricks/scripts/"))

Wrote 1045 bytes.


path,name,size,modificationTime
dbfs:/databricks/scripts/selenium-install.sh,selenium-install.sh,1045,1726529872000


In [ ]:
%sh
/dbfs/databricks/scripts/selenium-install.sh

/dbfs/databricks/scripts/selenium-install.sh: line 3: fg: no job control
######################################################################## 100.0%


Archive:  /tmp/chrome/1356175-chrome-linux.zip
  inflating: /tmp/chrome/1356175/chrome-linux/MEIPreload/manifest.json  
  inflating: /tmp/chrome/1356175/chrome-linux/MEIPreload/preloaded_data.pb  
  inflating: /tmp/chrome/1356175/chrome-linux/PrivacySandboxAttestationsPreloaded/manifest.json  
  inflating: /tmp/chrome/1356175/chrome-linux/PrivacySandboxAttestationsPreloaded/privacy-sandbox-attestations.dat  
  inflating: /tmp/chrome/1356175/chrome-linux/chrome  
  inflating: /tmp/chrome/1356175/chrome-linux/chrome-wrapper  
  inflating: /tmp/chrome/1356175/chrome-linux/chrome_100_percent.pak  
  inflating: /tmp/chrome/1356175/chrome-linux/chrome_200_percent.pak  
  inflating: /tmp/chrome/1356175/chrome-linux/chrome_crashpad_handler  
  inflating: /tmp/chrome/1356175/chrome-linux/chrome_sandbox  
  inflating: /tmp/chrome/1356175/chrome-linux/icudtl.dat  
  inflating: /tmp/chrome/1356175/chrome-linux/libEGL.so  
  inflating: /tmp/chrome/1356175/chrome-linux/libGLESv2.so  
  inflating: /t

######################################################################## 100.0%


Archive:  /tmp/chrome/1356175-chromedriver_linux64.zip
  inflating: /tmp/chrome/1356175/chromedriver_linux64/LICENSE.chromedriver  
  inflating: /tmp/chrome/1356175/chromedriver_linux64/chromedriver  
Get:1 https://repos.azul.com/zulu/deb stable InRelease [5,289 B]
Get:2 https://repos.azul.com/zulu/deb stable/main amd64 Packages [342 kB]
Get:3 https://repos.azul.com/zulu/deb stable/main arm64 Packages [198 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy InRelease [270 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,267 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy/universe amd64 Packages [17.5 MB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,030 kB]
Get:11 http://archive.ubuntu.com/ubuntu jamm

W: https://repos.azul.com/zulu/deb/dists/stable/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.


Reading package lists...
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  libgbm-dev
0 upgraded, 1 newly installed, 0 to remove and 70 not upgraded.
Need to get 9,546 B of archives.
After this operation, 100 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libgbm-dev amd64 23.2.1-1ubuntu3.1~22.04.2 [9,546 B]


debconf: delaying package configuration, since apt-utils is not installed


Fetched 9,546 B in 0s (27.7 kB/s)
Selecting previously unselected package libgbm-dev:amd64.
(Reading database ... 106459 files and directories currently installed.)
Preparing to unpack .../libgbm-dev_23.2.1-1ubuntu3.1~22.04.2_amd64.deb ...
Unpacking libgbm-dev:amd64 (23.2.1-1ubuntu3.1~22.04.2) ...
Setting up libgbm-dev:amd64 (23.2.1-1ubuntu3.1~22.04.2) ...


In [ ]:
%sh
pip install fake_useragent selenium

  Obtaining dependency information for fake_useragent from https://files.pythonhosted.org/packages/e4/99/60d8cf1b26938c2e0a57e232f7f15641dfcd6f8deda454d73e4145910ff6/fake_useragent-1.5.1-py3-none-any.whl.metadata
  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/b3/38/5aac23e57d61707f91914506902e621632de8dc56b60446459901469b9e2/selenium-4.24.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/1c/70/efa56ce2271c44a7f4f43533a0477e6854a0948e9f7b76491de1fd3be7c9/trio-0.26.2-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/48/be/a9ae5f50cad5b6f85bd2574c2c923730098530096e170c1ce7452394d7aa/trio_websocket-0.11.1-py3-none-any.whl.metadata
  Obtaining dependency information for websocket-client~=1.8 from https://files.pythonhosted.org/packages/5a/84/44687a29792a70e111c5c477230a72c4b957d88d16141199bf9acb


[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


## Depth Chart scraping and data cleaning

In [ ]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import pandas as pd

url = 'https://www.ourlads.com/ncaa-football-depth-charts'

ua = UserAgent()
userAgent = ua.random
headers = {'User-Agent': userAgent}
page = requests.get(url, headers = headers)
soup = BeautifulSoup(page.content, "html.parser")
team_links = []
team_link = soup.find_all('div', class_= "nfl-dc-mm-team-links")
for i in team_link:
    team_links.append('https://www.ourlads.com/ncaa-football-depth-charts/'+ i.a['href'])

team_links_depth_chart = team_links[1:]
team_links_roster = [url.replace('depth-chart.aspx', 'roster.aspx') for url in team_links]
team_links_roster = team_links_roster[1:]
master_df = []

def get_all_depth_chart(url):
    ua = UserAgent()
    userAgent = ua.random
    head = {'User-Agent': userAgent}
    page = requests.get(url, headers = head)
    soup = BeautifulSoup(page.content, "html.parser")
    team_name = soup.find('div', {'class': 'pt-team'}).text.strip()

    table = soup.find('table', class_='table table-bordered')
    # Extract the headers
    headers = [th.text.strip() for th in table.find_all('th')]
    rows = []
    for tr in table.find('tbody').find_all('tr'):
        # Each cell within the row
        cells = [td.text.strip() for td in tr.find_all('td')]
        rows.append(cells)

    df = pd.DataFrame(rows, columns=headers)
    df['team_name'] = team_name
    print(team_name)
    df['field_pos'] = df['Pos'].where(df['Pos'].isin(['Offense', 'Defense', 'Special Teams']))

    # Forward fill the 'field_pos' column to propagate values downwards
    df['field_pos'] = df['field_pos'].ffill()
    df = df[~df['Pos'].isin(['Offense', 'Defense', 'Special Teams', 'OFF','ST','DEF'])]
    df.rename(columns={'Player 1': '1st String','Player 2': '2nd String','Player 3': '3rd String','Player 4': '4th String','Player 5': '5th String','Pos':'Position','No.':'No1'}, inplace=True)
    no_counter = 2  # Start with 'No2'

    new_columns = []
    for col in df.columns:
        if col == 'No':
            new_columns.append(f'No{no_counter}')
            no_counter += 1
        else:
            new_columns.append(col)

    df.columns = new_columns

    df['1st String'] = df['No1'].astype(str) + ' ' + df['1st String']
    df['2nd String'] = df['No2'].astype(str) + ' ' + df['2nd String']
    df['3rd String'] = df['No3'].astype(str) + ' ' + df['3rd String']
    df['4th String'] = df['No4'].astype(str) + ' ' + df['4th String']
    df['5th String'] = df['No5'].astype(str) + ' ' + df['5th String']

    df = df.drop(columns=['No1', 'No2', 'No3','No4','No5'])


    df = df.melt(id_vars=['Position','field_pos','team_name'], 
                        value_vars=['1st String', '2nd String', '3rd String','4th String', '5th String'], 
                        var_name='String Order', 
                        value_name='Player Name')
    # Step 1: Extract the Player Number
    df['Player Number'] = df['Player Name'].str.extract(r'^(\d+)')

    # Step 2: Remove the Player Number from the Player Name
    df['Player'] = df['Player Name'].str.replace(r'^\d+\s+', '', regex=True)

    # Step 3: Split the Player Name into First Name and Other Names
    df[['First Name', 'Other Names']] = df['Player'].str.split(' ', n=1, expand=True)

    # Step 4: Drop the original 'Player' column as it's now split
    df = df.drop(columns=['Player Name','Player'])

    df['First Name']=df['First Name'].str.replace(',','')
    df[['Last Name', 'School year']] = df['Other Names'].str.split(' ', n=1, expand=True)
    df = df.drop(columns=['Other Names'])
    master_df.append(df)
    

for i in team_links_depth_chart:
    get_all_depth_chart(i)

df_depth_chart = pd.concat(master_df)
df_depth_chart = df_depth_chart.dropna(subset=['Player Number'])



Army Black Knights
Charlotte 49ers
East Carolina Pirates
Florida Atlantic Owls
Memphis Tigers
Navy Midshipmen
North Texas Mean Green
Rice Owls
South Florida Bulls
Temple Owls
Tulane Green Wave
Tulsa Golden Hurricane
UAB Blazers
UTSA Roadrunners
Boston College Eagles
California Golden Bears
Clemson Tigers
Duke Blue Devils
Florida State Seminoles
Georgia Tech Yellow Jackets
Louisville Cardinals
Miami Hurricanes
North Carolina Tar Heels
North Carolina State Wolfpack
Pittsburgh Panthers
SMU Mustangs
Stanford Cardinal
Syracuse Orange
Virginia Cavaliers
Virginia Tech Hokies
Wake Forest Demon Deacons
Illinois Fighting Illini
Indiana Hoosiers
Iowa Hawkeyes
Maryland Terrapins
Michigan Wolverines
Michigan State Spartans
Minnesota Golden Gophers
Nebraska Cornhuskers
Northwestern Wildcats
Ohio State Buckeyes
Oregon Ducks
Penn State Nittany Lions
Purdue Boilermakers
Rutgers Scarlet Knights
UCLA Bruins
USC Trojans
Washington Huskies
Wisconsin Badgers
Arizona Wildcats
Arizona State Sun Devils
Baylor 

## Data Quality for Depth Chart Data

In [ ]:
import os

# Create the directory if it does not exist
os.makedirs('/dbfs/FileStore', exist_ok=True)


In [ ]:
import pandas as pd
import numpy as np
import os

def run_data_quality_checks(df):
    # Initialize a list to store all check results
    check_results = []

    # Check for null values in 'Position', 'field_pos', and 'team_name'
    for column in ['Position', 'field_pos', 'team_name']:
        null_count = df[column].isnull().sum()
        check_results.append({
            'check': f"'{column}' not null",
            'result': 'Pass' if null_count == 0 else 'Fail',
            'details': f"{null_count} null values found"
        })

    # Check if 'Player Number' is an integer
    player_number_check = df['Player Number'].astype(str).str.match(r'^\d+$').all()
    check_results.append({
        'check': "'Player Number' is integer",
        'result': 'Pass' if player_number_check else 'Fail',
        'details': "All values are integers" if player_number_check else "Non-integer values found"
    })

    # Check if 'String Order' belongs to predefined values
    valid_string_orders = ['1st String', '2nd String', '3rd String', '4th String', '5th String']
    string_order_check = df['String Order'].isin(valid_string_orders).all()
    check_results.append({
        'check': "'String Order' in predefined set",
        'result': 'Pass' if string_order_check else 'Fail',
        'details': "All values are valid" if string_order_check else "Invalid values found"
    })

    # Check if 'field_pos' belongs to predefined values
    valid_field_pos = ['Defense', 'Offense', 'Special Teams']
    field_pos_check = df['field_pos'].isin(valid_field_pos).all()
    check_results.append({
        'check': "'field_pos' in predefined set",
        'result': 'Pass' if field_pos_check else 'Fail',
        'details': "All values are valid" if field_pos_check else "Invalid values found"
    })

    # Convert check results to a DataFrame for easy viewing
    results_df = pd.DataFrame(check_results)
    
    return results_df


results = run_data_quality_checks(df_depth_chart)

if (results['result'] == 'Pass').all():
    df_depth_chart.to_parquet('/dbfs/FileStore/depth_chart.parquet')
    print("All rows in the column are 'pass'.")
else:
    print(results)


All rows in the column are 'pass'.


## Roster scraping and data cleaning

In [ ]:
import numpy as np

all_df = []
def get_roster(url):
    ua = UserAgent()
    userAgent = ua.random
    head = {'User-Agent': userAgent}
    page = requests.get(url, headers = head)
    soup = BeautifulSoup(page.content, "html.parser")
    team_name = soup.find('div', {'class': 'pt-team'}).text.strip()

    table = soup.find('table', class_='table table-bordered')
    # Extract the headers
    headers = [th.text.strip() for th in table.find_all('th')]
    rows = []
    for tr in table.find('tbody').find_all('tr'):
        # Each cell within the row
        cells = [td.text.strip() for td in tr.find_all('td')]
        rows.append(cells)

    df = pd.DataFrame(rows, columns=headers)
    df = df[~df['Player'].isin(['Active Players'])]
    df.rename(columns={'#': 'Player Number','Pos.': 'Position','HT':'height','WT':'Weight'}, inplace=True)
    df[['First Name', 'Other Names']] = df['Player'].str.split(' ', n=1, expand=True)
    df['First Name']=df['First Name'].str.replace(',','')
    df['height'] = pd.to_numeric(df['height'])
    df['height']= (df['height'] /10).astype(int)
    df['height'] = df['height'].astype(str)

    df['height'] = df['height'].str.rstrip('0')
    single_digit_mask = df['height'].str.len() == 1
    df['formatted_height'] = np.where(single_digit_mask,
                                    df['height'] + "'",
                                    df['height'].str[0] + "'" + df['height'].str[1:])
    df['formatted_height']=df['formatted_height'].str.replace('0','')
    df['height'] = df['formatted_height']
    df = df.drop(columns=['Player','formatted_height'])
    df['team_name'] = team_name
    print(team_name)
    all_df.append(df)

for i in team_links_roster:
    get_roster(i)

df_roster = pd.concat(all_df)


df_roster['Weight'] = pd.to_numeric(df_roster['Weight'], errors='coerce')
df_roster['Player Number'] = pd.to_numeric(df_roster['Player Number'], errors='coerce')
df_roster

Army Black Knights
Charlotte 49ers
East Carolina Pirates
Florida Atlantic Owls
Memphis Tigers
Navy Midshipmen
North Texas Mean Green
Rice Owls
South Florida Bulls
Temple Owls
Tulane Green Wave
Tulsa Golden Hurricane
UAB Blazers
UTSA Roadrunners
Boston College Eagles
California Golden Bears
Clemson Tigers
Duke Blue Devils
Florida State Seminoles
Georgia Tech Yellow Jackets
Louisville Cardinals
Miami Hurricanes
North Carolina Tar Heels
North Carolina State Wolfpack
Pittsburgh Panthers
SMU Mustangs
Stanford Cardinal
Syracuse Orange
Virginia Cavaliers
Virginia Tech Hokies
Wake Forest Demon Deacons
Illinois Fighting Illini
Indiana Hoosiers
Iowa Hawkeyes
Maryland Terrapins
Michigan Wolverines
Michigan State Spartans
Minnesota Golden Gophers
Nebraska Cornhuskers
Northwestern Wildcats
Ohio State Buckeyes
Oregon Ducks
Penn State Nittany Lions
Purdue Boilermakers
Rutgers Scarlet Knights
UCLA Bruins
USC Trojans
Washington Huskies
Wisconsin Badgers
Arizona Wildcats
Arizona State Sun Devils
Baylor 

,Player Number,Position,height,Weight,Class,High School,Hometown,First Name,Other Names,team_name
1,90,DL,6'1,275,SO,Holy Cross,"San Antonio, TX",Ali,Amir,Army Black Knights
2,38,PT,6'1,213,JR,Independence,"Franklin, TN",Allan,Cooper,Army Black Knights
3,47,WR,6'1,205,FR,Edina,"Edina, MN",Anderson,Brady,Army Black Knights
4,58,OL,6'2,285,SO,Billy Ryan,"Denton, TX",Appleton,Henry,Army Black Knights
5,78,OL,6'5,317,FR,Lower Merion,"Ardmore, PA",Archawski,Xavier,Army Black Knights
...,...,...,...,...,...,...,...,...,...,...
122,5,DE/OLB,6'4,230,RS JR/TR,Cardinal Gibbons,"Carver Ranches, FL",Williams,Jah-Mal,Troy Trojans
123,4,RB,5'1,199,RS JR,Blount,"Mobile, AL",Williams,Jarris,Troy Trojans
124,86,TE,6'4,241,RS FR,The King's Academy,"West Palm Beach, FL",Worley,Jackson,Troy Trojans
125,9,CB,6'1,187,SR/TR,Orange Lutheran,"Long Beach, CA",Yancey,Damaje,Troy Trojans


## Data Quality for Roster Data

In [ ]:
import pandas as pd
import numpy as np

def run_roster_data_quality_checks(df):
    check_results = []

    # Check 'Player Number' is not null and is an integer
    player_number_null = df['Player Number'].isnull().sum()
    player_number_int = df['Player Number'].dtype == 'int64'
    check_results.extend([
        {
            'check': "'Player Number' not null",
            'result': 'Pass' if player_number_null == 0 else 'Fail',
            'details': f"{player_number_null} null values found"
        },
        {
            'check': "'Player Number' is integer",
            'result': 'Pass' if player_number_int else 'Fail',
            'details': f"Type is {df['Player Number'].dtype}"
        }
    ])

    # Check 'Position' is not null
    position_null = df['Position'].isnull().sum()
    check_results.append({
        'check': "'Position' not null",
        'result': 'Pass' if position_null == 0 else 'Fail',
        'details': f"{position_null} null values found"
    })

    # Check 'Weight' is integer and within range
    weight_int = df['Weight'].dtype == 'int64'
    weight_range = df['Weight'].between(100, 450).all()
    check_results.extend([
        {
            'check': "'Weight' is integer",
            'result': 'Pass' if weight_int else 'Fail',
            'details': f"Type is {df['Weight'].dtype}"
        },
        {
            'check': "'Weight' between 100 and 450",
            'result': 'Pass' if weight_range else 'Fail',
            'details': f"Range is {df['Weight'].min()} to {df['Weight'].max()}"
        }
    ])

    # Check 'team_name' is not null
    team_name_null = df['team_name'].isnull().sum()
    check_results.append({
        'check': "'team_name' not null",
        'result': 'Pass' if team_name_null == 0 else 'Fail',
        'details': f"{team_name_null} null values found"
    })

    # Convert check results to a DataFrame
    results_df = pd.DataFrame(check_results)
    return results_df

results = run_roster_data_quality_checks(df_roster)

if (results['result'] == 'Pass').all():
    df_roster.to_parquet('/dbfs/FileStore/roster.parquet')
    print("All rows in the column are 'pass'.")
else:
    print(results)

All rows in the column are 'pass'.


## Player stats scraping of ESPN with Selenium
## Data quality checks attached as well

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from fake_useragent import UserAgent
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import re
import pandas as pd



# URLs to scrape data from ESPN's college football player stats pages
urls = ['https://www.espn.com/college-football/stats/player',
        'https://www.espn.com/college-football/stats/player/_/stat/rushing',
        'https://www.espn.com/college-football/stats/player/_/stat/receiving',
        'https://www.espn.com/college-football/stats/player/_/view/scoring']

def get_espn_college_football_stats(url):
    s = Service('/tmp/chrome/latest/chromedriver_linux64/chromedriver')
    options = webdriver.ChromeOptions()
    options.binary_location = "/tmp/chrome/latest/chrome-linux/chrome"

    ua = UserAgent()  # Instantiate UserAgent to generate random user agents
    userAgent = ua.random  # Generate a random user-agent string
    options.add_argument(f'user-agent={userAgent}')  # Set the random user-agent string in browser options
    options.add_argument('--blink-settings=imagesEnabled=false')  
    options.add_argument('headless')
    options.add_argument('--disable-infobars')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--no-sandbox')
    options.add_argument('--remote-debugging-port=9222')
    options.add_argument('--homedir=/tmp/chrome/chrome-user-data-dir')
    options.add_argument('--user-data-dir=/tmp/chrome/chrome-user-data-dir')
    prefs = {"download.default_directory":"/tmp/chrome/chrome-user-data-di",
            "download.prompt_for_download":False
    }
    options.add_experimental_option("prefs",prefs)
    driver = webdriver.Chrome(service=s, options=options)

    driver.get(url)  # Load the specified URL in the Chrome browser
        
    time.sleep(5)  # Pause for 5 seconds to allow the page to load completely

    # Continuously click the "Show More" button until no more data can be loaded
    while True:
        try:
            # Locate the "Show More" button by its class name
            element = driver.find_element(By.CLASS_NAME, 'AnchorLink.loadMore__link')
            # Wait until the "Show More" button is clickable
            WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, 'AnchorLink.loadMore__link')))
            # Scroll the "Show More" button into view
            driver.execute_script("arguments[0].scrollIntoView();", element)
            # Click the "Show More" button to load more data
            driver.execute_script("arguments[0].click();", element)
            time.sleep(5)  # Pause for 5 seconds between clicks
        except Exception as e:  # Catch any exception if "Show More" button is not found
            print(e)  # Print the exception message
            break  # Exit the loop if no more data can be loaded
        print("Complete")  # Log progress when a "Show More" click is successful

    print('all show more clicked')  # Indicate that all data has been loaded

    page = driver.page_source  # Get the page source (HTML content)

    # Clean the HTML by removing unwanted rows (optional step)
    cleaned_html = re.sub(r'<tr class="Table__TR.Table__even">.*?</tr>', '', str(page), flags=re.DOTALL)

    # Parse the cleaned HTML content using BeautifulSoup
    soup = BeautifulSoup(cleaned_html, 'html.parser')

    tables = soup.find_all('table')  # Find all table elements in the HTML
    driver.quit()  # Close the Chrome browser

    dataframes = []  # List to store extracted data as DataFrames

    # Loop over each table found in the HTML and extract it into a pandas DataFrame
    for table in tables:
        headers = [th.text.strip() for th in table.find_all('th', class_='Table__TH')]  # Extract table headers
        headers = list(filter(None, headers))  # Filter out empty headers
        
        rows = []  # List to store rows of table data
        # Extract data from each row in the table
        for row in table.find_all('tr'):
            cells = [cell.text.strip() for cell in row.find_all('td')]  # Extract text from each table cell
            if cells:  # Avoid adding empty rows
                rows.append(cells)
        
        # Create a DataFrame if headers exist; otherwise, use default row indexing
        if headers:
            df = pd.DataFrame(rows, columns=headers)  # Create DataFrame with headers
        else:
            df = pd.DataFrame(rows)  # Create DataFrame without headers
        
        dataframes.append(df)  # Append the DataFrame to the list

    # Concatenate all the DataFrames for the current page into a single DataFrame
    dataframes = pd.concat(dataframes, axis=1)

    # Extract the team abbreviation from the 'Name' column
    dataframes['College'] = dataframes['Name'].str.extract(r'([A-Z]{2,})$')

    # Remove the uppercase letters (team abbreviations) from the 'Name' column
    dataframes['Name'] = dataframes['Name'].str.replace(r'([A-Z]{2,})$', '', regex=True).str.strip()
    dataframes.rename(columns={'POS': 'Position','RK': 'Rank'}, inplace=True)
    dataframes["Rank"] = pd.to_numeric(dataframes["Rank"])
    dataframes.loc[dataframes['Name'].str.contains('TA&M') & dataframes['College'].isnull(), 'College'] = 'TA&M'

    # Remove 'TA&M' from the 'name' column
    dataframes['Name'] = dataframes['Name'].str.replace('TA&M', '').str.strip()
    dataframes = dataframes[dataframes['Position'] != '-']

    df = dataframes
    check_results = []

    # Check 'Rk' (Rank) is not null and is an integer
    rk_null = df['Rank'].isnull().sum()
    rk_int = df['Rank'].dtype == 'int64'
    check_results.extend([
        {
            'check': "'Rank' not null",
            'result': 'Pass' if rk_null == 0 else 'Fail',
            'details': f"{rk_null} null values found"
        },
        {
            'check': "'Rank' is integer",
            'result': 'Pass' if rk_int else 'Fail',
            'details': f"Type is {df['Rank'].dtype}"
        }
    ])

    # Check 'Name' is not null and is a string
    name_null = df['Name'].isnull().sum()
    name_string = df['Name'].dtype == 'object'
    check_results.extend([
        {
            'check': "'Name' not null",
            'result': 'Pass' if name_null == 0 else 'Fail',
            'details': f"{name_null} null values found"
        },
        {
            'check': "'Name' is string",
            'result': 'Pass' if name_string else 'Fail',
            'details': f"Type is {df['Name'].dtype}"
        }
    ])

    # Check 'POS' is not null and is in a set of valid positions
    valid_positions = {
    "DL",
    "EDGE",
    "S",
    "QB",
    "RB",  # Running Back
    "FB",  # Fullback
    "WR",  # Wide Receiver
    "TE",  # Tight End
    "LT",  # Left Tackle
    "LG",  # Left Guard
    "C",   # Center
    "RG",  # Right Guard
    "RT",  # Right Tackle
    "DT",  # Defensive Tackle
    "DE",  # Defensive End
    "NT",  # Nose Tackle
    "LB",  # Linebacker
    "MLB", # Middle Linebacker
    "OLB", # Outside Linebacker
    "CB",  # Cornerback
    "FS",  # Free Safety
    "SS",  # Strong Safety
    "DB",  # Defensive Back (general term for CB and S)
    "OL",  # Offensive Line (general term for LT, LG, C, RG, RT)
    "PK",  # Placekicker
    "K",   # Kicker
    "P",   # Punter
    "LS",  # Long Snapper
    "KR",  # Kick Returner
    "PR",  # Punt Returner
    "G",   # Gunner (Special Teams)
    "H",   # Holder
    "KO"   # Kickoff Specialist
}  # Add more as needed
    pos_null = df['Position'].isnull().sum()
    pos_valid = df['Position'].isin(valid_positions).all()
    check_results.extend([
        {
            'check': "'Position' not null",
            'result': 'Pass' if pos_null == 0 else 'Fail',
            'details': f"{pos_null} null values found"
        },
        {
            'check': "'Position' is valid",
            'result': 'Pass' if pos_valid else 'Fail',
            'details': f"Invalid positions: {set(df['Position'].unique()) - valid_positions}"
        }
    ])

    # Check 'College' is not null and is a string
    College_null = df['College'].isnull().sum()
    College_string = df['College'].dtype == 'object'
    check_results.extend([
        {
            'check': "'College' not null",
            'result': 'Pass' if College_null == 0 else 'Fail',
            'details': f"{College_null} null values found"
        },
        {
            'check': "'College' is string",
            'result': 'Pass' if College_string else 'Fail',
            'details': f"Type is {df['College'].dtype}"
        }
    ])

    # Convert check results to a DataFrame
    results_df = pd.DataFrame(check_results)
    results_df = pd.DataFrame(check_results)
    if (results_df['result'] == 'Pass').all():
        df.to_parquet(f"/dbfs/FileStore/player_stats_{url.rstrip('/').split('/')[-1]}.parquet")
        print("All rows in the column are 'pass'.")
    else:
        print(results_df)

for i in urls:
    print(i)
    get_espn_college_football_stats(i)



https://www.espn.com/college-football/stats/player/_/view/scoring
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Complete
Message: no such element: Unable to locate element: {"method":"css selector","selector":".AnchorLink.loadMore__link"}
  (Session info: chrome=130.0.6723.0); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x55ab57b71332 <unknown>
#1 0x55ab57b6315e <unknown>
#2 0x55ab57612e97 <unknown>
#3 0x55ab5765ab13 <unknown>
#4 0x55ab5765adf1 <unknown>
#5 0x55ab5769f2f4 <unknown>
#6 0x55ab5767f1bd <unknown>
#7 0x55ab5764ffcf <unknown>
#8 0x55ab5769cfb7 <unknown>
#9 0x55ab5767ee33 <unknown>
#10 0x55ab5764f051 <unknown>
#11 0x55ab5764e149 <unknown>
#12 0x55ab5764ee2c <unknown>
#13 0x55ab57b1e9cf <unknown>
#14 0x55ab57b39959 <unknow

## Move all extracted parquet files to your blob storage container.

In [ ]:
from datetime import datetime
import os
import shutil

# Set up the blob storage account details
account_name = "testtech"
container_name = "testtech"
mount_point = "/mnt/testtech"
storage_key = ""

# The path to your Parquet files on DBFS
dbfs_parquet_path = 'dbfs:/FileStore'

# 1. Mount the blob storage container
def mount_blob_storage():
    if not any(mount.mountPoint == mount_point for mount in dbutils.fs.mounts()):
        dbutils.fs.mount(
            source = f"wasbs://{container_name}@{account_name}.blob.core.windows.net/",
            mount_point = mount_point,
            extra_configs = {f"fs.azure.account.key.{account_name}.blob.core.windows.net": storage_key}
        )
    print("Blob storage mounted.")

# 2. Create a new folder named with the current date
def create_new_folder():
    current_date = datetime.now().strftime("%Y-%m-%d")
    new_folder_path = os.path.join(mount_point, current_date)
    dbutils.fs.mkdirs(new_folder_path)
    print(f"New folder created: {new_folder_path}")
    return new_folder_path

# 3. Move Parquet files to the newly created folder
def move_parquet_files_to_blob(new_folder_path):
    files = dbutils.fs.ls(dbfs_parquet_path)
    
    for file in files:
        if file.path.endswith(".parquet"):
            dbutils.fs.mv(file.path, new_folder_path)
            print(f"Moved file {file.path} to {new_folder_path}")
    print("All Parquet files moved.")

# 4. Unmount the blob storage container
def unmount_blob_storage():
    dbutils.fs.unmount(mount_point)
    print("Blob storage unmounted.")

# 5. Remove the Parquet files from DBFS
def remove_parquet_files_from_dbfs():
    # List all files in the directory
    files = dbutils.fs.ls(dbfs_parquet_path)
    
    # Loop through the files and remove only the ones with '.parquet' extension
    for file in files:
        if file.name.endswith(".parquet"):
            dbutils.fs.rm(file.path)
            print(f"Removed Parquet file: {file.path}")

    print(f"All Parquet files removed from {dbfs_parquet_path}")

# Main function to run the whole process
def move_parquet_files():
    mount_blob_storage()  # Mount the blob storage
    new_folder_path = create_new_folder()  # Create a date-named folder
    move_parquet_files_to_blob(new_folder_path)  # Move files to the new folder
    unmount_blob_storage()  # Unmount the blob storage
    remove_parquet_files_from_dbfs()  # Delete parquet files from DBFS

# Run the process
move_parquet_files()


All Parquet files removed from dbfs:/FileStore


In [ ]:
files = dbutils.fs.ls('dbfs:/FileStore')
files

[FileInfo(path='dbfs:/FileStore/depth_chart.parquet', name='depth_chart.parquet', size=130382, modificationTime=1726530423000),
 FileInfo(path='dbfs:/FileStore/player_stats_player.parquet', name='player_stats_player.parquet', size=24218, modificationTime=1726530597000),
 FileInfo(path='dbfs:/FileStore/player_stats_receiving.parquet', name='player_stats_receiving.parquet', size=47534, modificationTime=1726530915000),
 FileInfo(path='dbfs:/FileStore/player_stats_rushing.parquet', name='player_stats_rushing.parquet', size=38008, modificationTime=1726530729000),
 FileInfo(path='dbfs:/FileStore/player_stats_scoring.parquet', name='player_stats_scoring.parquet', size=34965, modificationTime=1726532043000),
 FileInfo(path='dbfs:/FileStore/roster.parquet', name='roster.parquet', size=375152, modificationTime=1726530556000),
 FileInfo(path='dbfs:/FileStore/tables/', name='tables/', size=0, modificationTime=1726531786000)]

In [12]:
import requests
import json
from datetime import datetime
import pandas as pd

url = "https://site.api.espn.com/apis/site/v2/sports/football/college-football/scoreboard"
response = requests.get(url, timeout=10)
response.raise_for_status()  # Raise an exception for bad status codes
data = response.json()

matches = []
for event in data.get('events', []):
    match = {
        'id': event['id'],
        'name': event['name'],
        'shortName': event['shortName'],
        'status': event['status']['type']['name'],
        'date': datetime.strptime(event['date'], "%Y-%m-%dT%H:%MZ"),
        'homeTeam': event['competitions'][0]['competitors'][0]['team']['displayName'],
        'homeScore': int(event['competitions'][0]['competitors'][0]['score']),
        'awayTeam': event['competitions'][0]['competitors'][1]['team']['displayName'],
        'awayScore': int(event['competitions'][0]['competitors'][1]['score'])
    }
    matches.append(match)
df = pd.DataFrame(matches)
df

,id,name,shortName,status,date,homeTeam,homeScore,awayTeam,awayScore
0,401628485,Illinois Fighting Illini at Nebraska Cornhuskers,ILL @ NEB,STATUS_SCHEDULED,2024-09-21 00:00:00,Nebraska Cornhuskers,0,Illinois Fighting Illini,0
1,401628492,Marshall Thundering Herd at Ohio State Buckeyes,MRSH @ OSU,STATUS_SCHEDULED,2024-09-21 16:00:00,Ohio State Buckeyes,0,Marshall Thundering Herd,0
2,401635552,NC State Wolfpack at Clemson Tigers,NCSU @ CLEM,STATUS_SCHEDULED,2024-09-21 16:00:00,Clemson Tigers,0,NC State Wolfpack,0
3,401636875,Arkansas State Red Wolves at Iowa State Cyclones,ARST @ ISU,STATUS_SCHEDULED,2024-09-21 18:00:00,Iowa State Cyclones,0,Arkansas State Red Wolves,0
4,401628493,Kent State Golden Flashes at Penn State Nittan...,KENT @ PSU,STATUS_SCHEDULED,2024-09-21 19:30:00,Penn State Nittany Lions,0,Kent State Golden Flashes,0
5,401628489,USC Trojans at Michigan Wolverines,USC @ MICH,STATUS_SCHEDULED,2024-09-21 19:30:00,Michigan Wolverines,0,USC Trojans,0
6,401628365,UCLA Bruins at LSU Tigers,UCLA @ LSU,STATUS_SCHEDULED,2024-09-21 19:30:00,LSU Tigers,0,UCLA Bruins,0
7,401628979,Miami (OH) RedHawks at Notre Dame Fighting Irish,M-OH @ ND,STATUS_SCHEDULED,2024-09-21 19:30:00,Notre Dame Fighting Irish,0,Miami (OH) RedHawks,0
8,401635554,Georgia Tech Yellow Jackets at Louisville Card...,GT @ LOU,STATUS_SCHEDULED,2024-09-21 19:30:00,Louisville Cardinals,0,Georgia Tech Yellow Jackets,0
9,401644736,Buffalo Bulls at Northern Illinois Huskies,BUFF @ NIU,STATUS_SCHEDULED,2024-09-21 19:30:00,Northern Illinois Huskies,0,Buffalo Bulls,0
